In [1]:
from openai import OpenAI
from tqdm import tqdm
import json
import numpy as np
import random
from collections import Counter
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_chinese import Rouge
import jieba
import json
from bert_score import score
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from GeneralRequestProcessor import GeneralRequestProcessor

In [2]:
client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1",
)

In [3]:
# 读取和保存数据的函数
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"错误：找不到文件 {file_path}。请检查文件路径是否正确。")
        return None
    except json.JSONDecodeError:
        print(f"错误：无法解码文件 {file_path} 中的JSON数据。请确保这是一个有效的JSON文件。")
        return None
    
def save_to_json_file(data, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        print(f"数据成功保存到 {file_path}")
    except Exception as e:
        print(f"保存文件时发生错误: {e}")   
## 获取8-shot
def make_shot(origin_data=None):
    shot_8 = ""
    for index, single_data in enumerate(origin_data[4492:4500]):
        shot_ = f"【**案例{index}**：\n{single_data['conversations'][1]['value']}\n】"
        shot_8 += shot_
    return shot_8

# 读取原始数据并处理成需要的格式
def make_messages(origin_data=None):
    """
    将原始数据集转换为大模型微调所需数据格式的新数据集
    """
    messages = []
    Real_outputs = []
    shot_8 = make_shot(origin_data)
    for index, single_data in enumerate(origin_data):
        random_number = random.randint(0, 9)
        message = [
            {"role": "system", "content": ""},
#             {"role": "user", "content": single_data['conversations'][0]['value']}
            {"role": "user", "content": "{}\n-{},以下是供你参考的人文社科领域学术文本摘要案例\n{}".format(head_template_COT3, single_data['conversations'][0]['value'], shot_8)}
        ]
        messages.append([message,index])
        Real_outputs.append(single_data['conversations'][1]['value'])
            
    return messages,Real_outputs

def ChatCompletions(message):
    response = client.chat.completions.create(
        model="MyModel",
        messages=message,
        stream=False,
        temperature=0.01,
        max_tokens=5000,
    )
    return response.choices[0].message.content        

def process_cot(abstract, model):
    abstract_COT = []
    if model == 'qwen':
        count = []
        for idx, value in enumerate(abstract):
            try:
                Model_output_COT = value.split('<最终摘要>')[1].strip('\n')
            except:
                count.append(idx)
                Model_output_COT = value
            abstract_COT.append(Model_output_COT)
        print(f"本次存在问题的id为{count}", '\n', '------------------------------')
    elif model == 'llama':
        count = []
        for idx, value in enumerate(abstract):
            try:
                Model_output_COT = value.split('最终摘要')[1].strip('\n')
            except:
                count.append(idx)
                Model_output_COT = value
            abstract_COT.append(Model_output_COT)
        print(f"本次存在问题的id为{count}", '\n', '------------------------------')
    else:
        print("输入有误")
    return abstract_COT

def process_deep(abstract):
    abstract_new = []
    idx_list = []
    count = []
    for idx, value in enumerate(abstract):
        try:
            Model_output_deepseek = value.split('</think>')[1].strip('\n')
        except:
            Model_output_deepseek = "文本生成失败"
        # 测试长度
        encoded_text = tokenizer.encode(value)
        if len(encoded_text) > 3000:
            Model_output_deepseek = "文本生成失败"
            count.append(idx)
        else:
            idx_list.append(idx)
        abstract_new.append(Model_output_deepseek)
    print(f"本次存在问题的id为{count}", '\n', '------------------------------')
    return abstract_new, idx_list

head_template = [
    "请你根据给出的学术全文本内容，生成关于该文本的摘要。",
    "请根据提供的学术文本内容，生成一个该文本的摘要。",
    "请根据给定的学术全文，撰写一个简要总结。",
    "请根据所提供的学术文章内容，生成一个概述。",
    "你需要阅读以下学术文本，并根据其内容编写摘要",
    "根据给出的学术材料，请你整理出一份摘要。",
    "请将下面的学术文本内容总结成一个简短的摘要。",
    "请从提供的学术文本中提炼出摘要内容。",
    "请根据以下学术文本写一段简短的总结。",
    "请基于下列学术全文，提炼并生成相关摘要。"
]
head_template_COT = """
下面，我将会给你一份学术文本内容，你需要分析给出的内容，编写关于该学术文本的摘要。除去学术文本，我还会给你其他学术文本的摘要作为参考案例
## 编写摘要要求
- 首先你需要分析我给你的学术文本摘要案例，案例总共8个，你需要分析每一份摘要案例的行文特点和语言特征，作为你生成摘要的支撑。
- 然后你需要分析我所给你的学术文本主要讲述的核心内容，涉及的实验对象等学术信息，并分析你认为的每一个段落的关键语句。
- 最后，你需要根据你的分析，结合你从我所提供的摘要的行文特点和语言特征，编写一份我所给你的学术文本内容的摘要。
## 输出格式
- 你输出数据的格式应严格为：<所给摘要特征总结>xxxxx\n<学术文本内容分析>xxxxx\n<最终摘要>xxxxx
"""

head_template_COT2 = """
下面，我将会给你一份学术文本内容，你需要分析给出的内容，编写关于该学术文本的摘要。除去学术文本，我还会给你其他学术文本的摘要作为参考案例
## 编写摘要要求
- 首先你需要分析我给你的学术文本摘要案例，案例总共8个，你需要分析每一份摘要案例的行文特点和语言特征，作为你生成摘要的支撑。
- 然后你需要分析我所给你的学术文本主要讲述的核心内容，涉及的实验对象等学术信息，并分析你认为的每一个段落的关键语句。
- 最后，你需要根据你的分析，结合你从我所提供的摘要的行文特点和语言特征，编写一份我所给你的学术文本内容的摘要。
## 输出格式
- 你的最终输出只需要给出你的摘要即可，不用给出其他内容！
"""

head_template_COT3 = """
下面，我会先给你一份人文社科学术文本，你需要总结该文本摘要。此外，我还会给你8个摘要案例作为编写学术文本的参考，编写中需要注意的点如下：
## 所给摘要特征总结要求
- 你需要分析我给你的学术文本摘要案例，案例总共8个，你需要分析每一份摘要案例的行文特点和语言特征，作为你生成摘要的支撑。
- 你要根据你对案例摘要分析的内容，在<所给摘要特征总结>模块给出人文社科领域学术文本摘要的行文特点和语言特征，为之后修改摘要提供依据。
## 学术文本内容分析要求
- 人文社科领域摘要的重点在于背景描述和本文论点的结合。
- 首先你需要识别出本文的研究对象，并在<学术文本内容分析>模块中以【研究对象：xxxx】的格式给出。
- 其次你需要识别出本文的研究背景，并在<学术文本内容分析>模块中以【研究背景：xxxx】的格式给出。
- 然后你需要以文章段落为单位进行分析，找出文中关于研究对象的论述，并在<学术文本内容分析>模块中以【论述x：xxxx】的格式给出。
- 你需要分析不同论述之间是并列、递进亦或对比等关系，并在<学术文本内容分析>模块中按照论述关系三元组【论述m，关系，论述n】的格式给出。
- 最后你要根据总结的论述关系三元组，分析出本文论述的核心论点，并在<学术文本内容分析>模块中以【核心论点：xxxx】的格式给出
## 初步摘要的编写
- 你要围绕<学术文本内容分析>模块中给出的“研究对象”、“研究背景”以及“核心论点”部分内容编写初步摘要，并在<初步摘要>模块给出。
## 摘要审改的编写
- 根据<初步摘要>模块的初步摘要，结合<所给摘要特征总结>模块中人文社科领域学术文本的行文特点和语言特征，以及你得到的8个案例，对初步摘要进行修改。
- 你需要比对初步摘要同案例摘要在语言特征方面的区别，并在<摘要审改>模块中以【区别x:xxxxx】形式给出。
- 你需要根据你所罗列的区别信息，在<摘要审改>模块中给出摘要修改的思路。
## 最终摘要的编写
- 根据在<摘要审改>模块中得到的思路，对<初步摘要>模块中的摘要内容进行修改，力求修改后的语言和结构特征同真实摘要一致。
- 真实摘要中较少使用（首先、其次、再次）这种前后顺序的连接词，因此在最终摘要编写过程中要注意该语言表达特点。
- 注意最终摘要中的组织语句主语、谓语宾语等结构的语言表达习惯要同真实摘要相一致。在<最终摘要>模块中给出你所转写的最终摘要。
## 输出格式
- 你输出数据的格式应严格为：<所给摘要特征总结>xxxxx\n<学术文本内容分析>xxxxx\n<初步摘要>xxxxx\n<摘要审改>xxxxx\n<最终摘要>xxxxx
- <最终摘要>模块仅需要给出你修改好的最终摘要，不要再输出其他的内容！
"""

In [168]:
tokenizer = AutoTokenizer.from_pretrained('/gemini/data-2/DeepSeek-R1-Distill-llama-8B/')
# 使用函数读取JSON文件
file_path = '/gemini/code/Abstract_extract/ProcessedData/rdfybk/train_data_abstract_mulIns_small.json'  # 将这里的文件路径替换为你自己的JSON文件路径
train_test_data = read_json_file(file_path)
# 获取输入数据
messages,Real_outputs = make_messages(train_test_data)

# 读取所有的摘要
abstract1 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B.json")
abstract2 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_cot.json")
abstract3 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_1.2.json")
abstract4 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B.json")
abstract5 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B_cot2.json")
abstract6 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B_1.4.json")
abstract7 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_8B.json")
abstract8 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_8B_cot2.json")
abstract9 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_7B.json")
abstract10 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/deepseek_outputs_7B_cot.json")
# 新补充的实验
abstract11 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_cot_plus2.json")
abstract12 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_7B_2.1.json")
abstract13 = read_json_file("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_8B_cot_plus.json")

In [169]:
abstract2_COT = process_cot(abstract2, 'qwen')
abstract5_COT = process_cot(abstract5, 'llama')
abstract7_new, idx_list = process_deep(abstract7)
abstract8_new, idx_list = process_deep(abstract8)
abstract9_new, idx_list = process_deep(abstract9)
abstract10_new, idx_list = process_deep(abstract10)
abstract11_COT = process_cot(abstract11, 'qwen')
abstract12_COT = process_cot(abstract12, 'qwen')
abstract13_COT = process_cot(abstract13, 'llama')

本次存在问题的id为[21, 26, 141, 229, 232, 239, 270, 335, 357, 497] 
 ------------------------------
本次存在问题的id为[0, 40, 62, 81, 83, 103, 178, 191, 195, 219, 223, 225, 361, 371, 387, 395, 412, 436, 452, 469, 477, 487, 489, 497, 499] 
 ------------------------------
本次存在问题的id为[21, 25, 26, 33, 36, 42, 106, 126, 128, 135, 141, 145, 155, 169, 198, 205, 215, 224, 229, 232, 239, 266, 270, 271, 295, 298, 320, 333, 335, 367, 385, 420, 443, 469, 477, 489, 497] 
 ------------------------------
本次存在问题的id为[2, 35, 83, 169, 229, 464, 469, 476, 482, 488, 496, 497, 499] 
 ------------------------------
本次存在问题的id为[357] 
 ------------------------------
本次存在问题的id为[36, 139, 486, 497] 
 ------------------------------
本次存在问题的id为[] 
 ------------------------------
本次存在问题的id为[17, 65, 120, 136, 211, 213, 273, 277, 328, 331, 369, 378, 390, 407, 409, 466, 497] 
 ------------------------------
本次存在问题的id为[75, 91, 153, 157, 186, 187, 229, 271, 372, 407, 409, 427, 470, 478] 
 ------------------------------


In [14]:
Evalution_Prompt_1 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的参考摘要，以及不同语言模型生成的学术文本摘要，请你以参考摘要为基准，从给出的摘要中选出逻辑和内容最为连贯的摘要，并只需要输出摘要的序号
- 参考摘要为:【{}】
- 待选择摘要1：【{}】
- 待选择摘要2：【{}】
- 待选择摘要3：【{}】
- 待选择摘要4：【{}】
- 待选择摘要5：【{}】
- 待选择摘要6：【{}】
- 待选择摘要7：【{}】
- 待选择摘要8：【{}】
- 待选择摘要9：【{}】
- 待选择摘要10：【{}】
- 待选择摘要11：【{}】
- 待选择摘要12：【{}】
你所选择的摘要序号为：
"""
Evalution_Prompt_2 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的原始摘要，以及不同语言模型生成的学术文本摘要，请你以参考摘要为基准，从给出的摘要中选出表达最流畅的摘要，并只需要输出摘要的序号
- 参考摘要为:【{}】
- 待选择摘要1：【{}】
- 待选择摘要2：【{}】
- 待选择摘要3：【{}】
- 待选择摘要4：【{}】
- 待选择摘要5：【{}】
- 待选择摘要6：【{}】
- 待选择摘要7：【{}】
- 待选择摘要8：【{}】
- 待选择摘要9：【{}】
- 待选择摘要10：【{}】
- 待选择摘要11：【{}】
- 待选择摘要12：【{}】
你所选择的摘要序号为：
"""
Evalution_Prompt_3 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的原始摘要，以及不同语言模型生成的学术文本摘要，请你以参考摘要为基准，从给出的摘要中选出同参考摘要最相似且在语言和结构表述上最一致的摘要，并只需要输出摘要的序号
- 参考摘要为:【{}】
- 待选择摘要1：【{}】
- 待选择摘要2：【{}】
- 待选择摘要3：【{}】
- 待选择摘要4：【{}】
- 待选择摘要5：【{}】
- 待选择摘要6：【{}】
- 待选择摘要7：【{}】
- 待选择摘要8：【{}】
- 待选择摘要9：【{}】
- 待选择摘要10：【{}】
- 待选择摘要11：【{}】
- 待选择摘要12：【{}】
你所选择的摘要序号为：
"""
Evalution_Prompt_4 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的原始摘要，以及不同语言模型生成的学术文本摘要，请你以参考摘要为基准，从给出的摘要中选出同原始摘要表述最一致、自身逻辑和内容连贯且语句最流畅的摘要，并只需要输出摘要的序号
- 参考摘要为:【{}】
- 待选择摘要1：【{}】
- 待选择摘要2：【{}】
- 待选择摘要3：【{}】
- 待选择摘要4：【{}】
- 待选择摘要5：【{}】
- 待选择摘要6：【{}】
- 待选择摘要7：【{}】
- 待选择摘要8：【{}】
- 待选择摘要9：【{}】
- 待选择摘要10：【{}】
- 待选择摘要11：【{}】
- 待选择摘要12：【{}】
你所选择的摘要序号为：
"""

In [15]:
choises_messages = []
for idx in range(len(abstract10_new)):
    message = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": Evalution_Prompt_4.format(Real_outputs[4500+idx], abstract1[idx], 
                                                              abstract2_COT[idx], abstract3[idx], abstract4[idx], abstract5_COT[idx], 
                                                              abstract6[idx], abstract7_new[idx], abstract8_new[idx],
                                                              abstract9_new[idx], abstract10_new[idx], abstract11_COT[idx],
                                                             abstract13_COT[idx])}
    ]
    choises_messages.append([message,idx])
len(choises_messages)

500

In [16]:
processor = GeneralRequestProcessor(max_concurrent_requests=100)
processor.apply(handle_function=ChatCompletions, data_list=choises_messages)
processor.process_all_requests()
results = processor.get_results()

2025-04-02 21:35:18,511 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,513 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,528 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,529 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,539 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,545 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,546 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,546 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,547 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,547 - HTTP Request: POST http://loc

2025-04-02 21:35:18,707 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,721 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,731 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,737 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,738 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,739 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,740 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,748 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,749 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:35:18,750 - HTTP Request: POST http://loc

2025-04-02 21:35:20,692 - Processed 47/500 requests. 53 currently processing. 453 remaining. Estimated time left: 1581.99 seconds.
2025-04-02 21:35:20,736 - Processed 48/500 requests. 52 currently processing. 452 remaining. Estimated time left: 1546.03 seconds.
2025-04-02 21:35:20,770 - Processed 49/500 requests. 51 currently processing. 451 remaining. Estimated time left: 1511.44 seconds.
2025-04-02 21:35:20,788 - Processed 50/500 requests. 50 currently processing. 450 remaining. Estimated time left: 1478.09 seconds.
2025-04-02 21:35:20,848 - Processed 51/500 requests. 49 currently processing. 449 remaining. Estimated time left: 1446.41 seconds.
2025-04-02 21:35:20,923 - Processed 52/500 requests. 48 currently processing. 448 remaining. Estimated time left: 1416.08 seconds.
2025-04-02 21:35:20,955 - Processed 53/500 requests. 47 currently processing. 447 remaining. Estimated time left: 1386.54 seconds.
2025-04-02 21:35:21,001 - Processed 54/500 requests. 46 currently processing. 446 r

2025-04-02 21:38:01,761 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,762 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,763 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,764 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,765 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,766 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,767 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,770 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,771 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,772 - HTTP Request: POST http://loc

2025-04-02 21:38:01,848 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,855 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,856 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,856 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,857 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,858 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,859 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,859 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,860 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:38:01,870 - HTTP Request: POST http://loc

...

2025-04-02 21:43:28,086 - Processed 308/500 requests. 94 currently processing. 192 remaining. Estimated time left: 406.15 seconds.
2025-04-02 21:43:28,156 - Processed 309/500 requests. 93 currently processing. 191 remaining. Estimated time left: 402.77 seconds.
2025-04-02 21:43:28,179 - Processed 310/500 requests. 92 currently processing. 190 remaining. Estimated time left: 399.38 seconds.
2025-04-02 21:43:28,227 - Processed 311/500 requests. 91 currently processing. 189 remaining. Estimated time left: 396.03 seconds.
2025-04-02 21:43:28,293 - Processed 312/500 requests. 90 currently processing. 188 remaining. Estimated time left: 392.71 seconds.
2025-04-02 21:43:28,369 - Processed 313/500 requests. 89 currently processing. 187 remaining. Estimated time left: 389.42 seconds.
2025-04-02 21:43:28,417 - Processed 314/500 requests. 88 currently processing. 186 remaining. Estimated time left: 386.13 seconds.
2025-04-02 21:43:28,456 - Processed 315/500 requests. 87 currently processing. 185 

2025-04-02 21:43:31,130 - Processed 371/500 requests. 32 currently processing. 129 remaining. Estimated time left: 227.60 seconds.
2025-04-02 21:43:31,163 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:43:31,188 - Processed 372/500 requests. 31 currently processing. 128 remaining. Estimated time left: 225.25 seconds.
2025-04-02 21:43:31,274 - Processed 373/500 requests. 30 currently processing. 127 remaining. Estimated time left: 222.92 seconds.
2025-04-02 21:43:31,308 - Processed 374/500 requests. 29 currently processing. 126 remaining. Estimated time left: 220.58 seconds.
2025-04-02 21:43:31,354 - Processed 375/500 requests. 28 currently processing. 125 remaining. Estimated time left: 218.27 seconds.
2025-04-02 21:43:31,400 - Processed 376/500 requests. 27 currently processing. 124 remaining. Estimated time left: 215.96 seconds.
2025-04-02 21:43:31,481 - Processed 377/500 requests. 26 currently processing. 123 remaining. Estimated time 

2025-04-02 21:46:09,502 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,503 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,503 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,504 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,510 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,513 - Processed 404/500 requests. 96 currently processing. 96 remaining. Estimated time left: 193.18 seconds.
2025-04-02 21:46:09,513 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,514 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,521 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-02 21:46:09,522 - HTTP

2025-04-02 21:46:10,142 - Processed 418/500 requests. 82 currently processing. 82 remaining. Estimated time left: 159.60 seconds.
2025-04-02 21:46:10,181 - Processed 419/500 requests. 81 currently processing. 81 remaining. Estimated time left: 157.29 seconds.
2025-04-02 21:46:10,212 - Processed 420/500 requests. 80 currently processing. 80 remaining. Estimated time left: 154.98 seconds.
2025-04-02 21:46:10,266 - Processed 421/500 requests. 79 currently processing. 79 remaining. Estimated time left: 152.69 seconds.
2025-04-02 21:46:10,326 - Processed 422/500 requests. 78 currently processing. 78 remaining. Estimated time left: 150.41 seconds.
2025-04-02 21:46:10,377 - Processed 423/500 requests. 77 currently processing. 77 remaining. Estimated time left: 148.14 seconds.
2025-04-02 21:46:10,427 - Processed 424/500 requests. 76 currently processing. 76 remaining. Estimated time left: 145.88 seconds.
2025-04-02 21:46:10,480 - Processed 425/500 requests. 75 currently processing. 75 remainin

2025-04-02 21:46:13,193 - Processed 482/500 requests. 18 currently processing. 18 remaining. Estimated time left: 30.50 seconds.
2025-04-02 21:46:13,231 - Processed 483/500 requests. 17 currently processing. 17 remaining. Estimated time left: 28.74 seconds.
2025-04-02 21:46:13,270 - Processed 484/500 requests. 16 currently processing. 16 remaining. Estimated time left: 27.00 seconds.
2025-04-02 21:46:13,310 - Processed 485/500 requests. 15 currently processing. 15 remaining. Estimated time left: 25.26 seconds.
2025-04-02 21:46:13,370 - Processed 486/500 requests. 14 currently processing. 14 remaining. Estimated time left: 23.53 seconds.
2025-04-02 21:46:13,402 - Processed 487/500 requests. 13 currently processing. 13 remaining. Estimated time left: 21.80 seconds.
2025-04-02 21:46:13,451 - Processed 488/500 requests. 12 currently processing. 12 remaining. Estimated time left: 20.09 seconds.
2025-04-02 21:46:13,490 - Processed 489/500 requests. 11 currently processing. 11 remaining. Esti

In [17]:
model_output = [""] * len(results)
for result in results:
    model_output[int(result['input'][-1])] = result['output']
Model_choises = model_output
save_to_json_file(Model_choises, '/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_new_choise8.json')

数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_new_choise8.json


In [18]:
Produced_choises = []
for index, choise in enumerate(Model_choises):
    try:
        number_choise = int(choise)
    except:
        number_choise = 0
        print(f"第{index}条存在问题，请人工检查")
    Produced_choises.append(number_choise)

In [19]:
element_count = Counter(Produced_choises)

# 输出统计结果
for element, count in element_count.items():
    print(f"元素 '{element}' 出现了 {count} 次")

元素 '2' 出现了 160 次
元素 '7' 出现了 14 次
元素 '3' 出现了 153 次
元素 '11' 出现了 57 次
元素 '6' 出现了 29 次
元素 '4' 出现了 22 次
元素 '5' 出现了 15 次
元素 '1' 出现了 28 次
元素 '12' 出现了 11 次
元素 '8' 出现了 6 次
元素 '10' 出现了 3 次
元素 '9' 出现了 2 次


## 拆分评分

In [199]:
Evalution_Prompt_1 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的参考摘要，以及不同语言模型生成的学术文本摘要，考虑同参考摘的文本相似性，摘要自身逻辑和内容连贯性，对摘要进行评分
- 参考摘要为:【{}】
- 待评分摘要1：【{}】
- 待评分摘要2：【{}】
- 待评分摘要3：【{}】
- 待评分摘要4：【{}】
- 待评分摘要5：【{}】
- 输出数据个格式为<思考>xxxx</思考>\n\n<评分>x</评分><最高分序号>x</最高分序号>
- <思考>部分给出你对每个摘要评分的原因。
- 其中<评分>中按照【"摘要1"：评分, "摘要2"：评分, "摘要3"：评分, "摘要4"：评分, "摘要5"：评分】的json格式输出。
- 最后在<最高分序号>中，给出你所评的评分最高的摘要对应的需要，仅需要给出序号即可。
"""
Evalution_Prompt_2 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的原始摘要，以及不同语言模型生成的学术文本摘要，考虑同参考摘的文本相似性，摘要自身语言流畅性，对摘要进行评分
- 参考摘要为:【{}】
- 待评分摘要1：【{}】
- 待评分摘要2：【{}】
- 待评分摘要3：【{}】
- 待评分摘要4：【{}】
- 待评分摘要5：【{}】
- 输出数据个格式为<思考>xxxx</思考>\n\n<评分>x</评分><最高分序号>x</最高分序号>
- <思考>部分给出你对每个摘要评分的原因。
- 其中<评分>中按照【"摘要1"：评分, "摘要2"：评分, "摘要3"：评分, "摘要4"：评分, "摘要5"：评分】的json格式输出。
- 最后在<最高分序号>中，给出你所评的评分最高的摘要对应的需要，仅需要给出序号即可。
"""
Evalution_Prompt_3 = """
你是一位优秀的摘要评估专家，现在我将给你一段关于学术文献的原始摘要，以及不同语言模型生成的学术文本摘要，考虑同参考摘的文本相似性，同参考摘要在内容一致性，对摘要进行评分
- 参考摘要为:【{}】
- 待评分摘要1：【{}】
- 待评分摘要2：【{}】
- 待评分摘要3：【{}】
- 待评分摘要4：【{}】
- 待评分摘要5：【{}】
- 输出数据个格式为<思考>xxxx</思考>\n\n<评分>x</评分><最高分序号>x</最高分序号>
- <思考>部分给出你对每个摘要评分的原因。
- 其中<评分>中按照【"摘要1"：评分, "摘要2"：评分, "摘要3"：评分, "摘要4"：评分, "摘要5"：评分】的json格式输出。
- 最后在<最高分序号>中，给出你所评的评分最高的摘要对应的需要，仅需要给出序号即可。
"""
Evalution_Prompt_4 = """
你是一位优秀的摘要评估专家，现在我将给你参考摘要摘要以及不同的模型生成摘要，考虑同参考摘要语言表达相似性，同参考摘要内容一致性、摘要自身逻辑连贯性和语句流畅性，对摘要进行评分，考虑到摘要的精炼性，其中若待评分摘要相对参考摘要存在扩展，需要进行降分，扩展程度越高，降分越多。
- 参考摘要为:【{}】
- 待评分摘要1：【{}】
- 待评分摘要2：【{}】
- 待评分摘要3：【{}】
- 待评分摘要4：【{}】
- 待评分摘要5：【{}】
- 输出数据个格式为<思考>xxxx</思考>\n\n<评分>x</评分><最高分序号>x</最高分序号>
- <思考>部分给出你对每个摘要评分的原因。
- 其中<评分>中按照【"摘要1"：评分, "摘要2"：评分, "摘要3"：评分, "摘要4"：评分, "摘要5"：评分】的json格式输出。
- 最后在<最高分序号>中，给出你所评的评分最高的摘要对应的需要，仅需要给出序号即可。
"""

In [200]:
choises_messages = []
for idx in range(len(abstract10_new)):
    message = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": Evalution_Prompt_1.format(Real_outputs[4500+idx] ,abstract1[idx], 
                                                              abstract2_COT[idx], abstract11_COT[idx] ,abstract3[idx], abstract10_new[idx])}
    ]
    choises_messages.append([message,idx])
len(choises_messages)

500

In [211]:
choises_messages = []
for idx in range(len(abstract10_new)):
    message = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": Evalution_Prompt_4.format(Real_outputs[4500+idx], abstract4[idx], 
                                                              abstract5_COT[idx], abstract13_COT[idx],abstract6[idx],
                                                              abstract8_new[idx])}
    ]
    choises_messages.append([message,idx])
len(choises_messages)

500

In [212]:
processor = GeneralRequestProcessor(max_concurrent_requests=100)
processor.apply(handle_function=ChatCompletions, data_list=choises_messages)
processor.process_all_requests()
results = processor.get_results()

2025-04-03 15:09:13,242 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:13,277 - Processed 1/500 requests. 99 currently processing. 499 remaining. Estimated time left: 54795.54 seconds.
2025-04-03 15:09:14,476 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:14,501 - Processed 2/500 requests. 99 currently processing. 498 remaining. Estimated time left: 27647.48 seconds.
2025-04-03 15:09:14,585 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:14,612 - Processed 3/500 requests. 99 currently processing. 497 remaining. Estimated time left: 18413.15 seconds.
2025-04-03 15:09:17,289 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:17,324 - Processed 4/500 requests. 99 currently processing. 496 remaining. Estimated time left: 14118.30 seconds.
2025-04-03 15:09:17,402 - HTTP Request: POST http://loca

2025-04-03 15:09:51,816 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:51,838 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:51,871 - Processed 36/500 requests. 99 currently processing. 464 remaining. Estimated time left: 1912.77 seconds.
2025-04-03 15:09:51,920 - Processed 37/500 requests. 98 currently processing. 463 remaining. Estimated time left: 1857.67 seconds.
2025-04-03 15:09:51,929 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:52,051 - Processed 38/500 requests. 99 currently processing. 462 remaining. Estimated time left: 1806.47 seconds.
2025-04-03 15:09:53,186 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:09:53,210 - Processed 39/500 requests. 99 currently processing. 461 remaining. Estimated time left: 1770.04 seconds.
2025-04-03 15:09:55,174 - HTTP Request: POST http://loca

2025-04-03 15:10:23,681 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:10:23,715 - Processed 71/500 requests. 99 currently processing. 429 remaining. Estimated time left: 1089.11 seconds.
2025-04-03 15:10:23,793 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:10:23,826 - Processed 72/500 requests. 99 currently processing. 428 remaining. Estimated time left: 1072.14 seconds.
2025-04-03 15:10:24,114 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:10:24,164 - Processed 73/500 requests. 99 currently processing. 427 remaining. Estimated time left: 1056.95 seconds.
2025-04-03 15:10:26,525 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:10:26,548 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:10:26,580 - Processed 74/500 requests. 99 currently processing. 426 

2025-04-03 15:11:15,441 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:11:15,486 - Processed 106/500 requests. 99 currently processing. 394 remaining. Estimated time left: 862.41 seconds.
2025-04-03 15:11:17,266 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:11:17,306 - Processed 107/500 requests. 99 currently processing. 393 remaining. Estimated time left: 858.87 seconds.
2025-04-03 15:11:17,695 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:11:17,744 - Processed 108/500 requests. 99 currently processing. 392 remaining. Estimated time left: 850.34 seconds.
2025-04-03 15:11:18,952 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:11:19,002 - Processed 109/500 requests. 99 currently processing. 391 remaining. Estimated time left: 844.90 seconds.
2025-04-03 15:11:20,243 - HTTP Request: POST http://loca

2025-04-03 15:11:57,025 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:11:57,054 - Processed 141/500 requests. 99 currently processing. 359 remaining. Estimated time left: 696.58 seconds.
2025-04-03 15:12:01,104 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:12:01,105 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:12:01,159 - Processed 142/500 requests. 99 currently processing. 358 remaining. Estimated time left: 700.10 seconds.
2025-04-03 15:12:01,203 - Processed 143/500 requests. 98 currently processing. 357 remaining. Estimated time left: 693.37 seconds.
2025-04-03 15:12:02,755 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:12:02,804 - Processed 144/500 requests. 99 currently processing. 356 remaining. Estimated time left: 690.59 seconds.
2025-04-03 15:12:03,891 - HTTP Request: POST http://loca

...

2025-04-03 15:16:10,897 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:16:10,937 - Processed 351/500 requests. 99 currently processing. 149 remaining. Estimated time left: 223.91 seconds.
2025-04-03 15:16:11,961 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:16:12,015 - Processed 352/500 requests. 99 currently processing. 148 remaining. Estimated time left: 222.23 seconds.
2025-04-03 15:16:13,793 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:16:13,847 - Processed 353/500 requests. 99 currently processing. 147 remaining. Estimated time left: 220.87 seconds.
2025-04-03 15:16:13,912 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:16:13,946 - Processed 354/500 requests. 99 currently processing. 146 remaining. Estimated time left: 218.79 seconds.
2025-04-03 15:16:16,290 - HTTP Request: POST http://loca

2025-04-03 15:16:59,131 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:16:59,176 - Processed 386/500 requests. 99 currently processing. 114 remaining. Estimated time left: 170.03 seconds.
2025-04-03 15:16:59,249 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:16:59,273 - Processed 387/500 requests. 99 currently processing. 113 remaining. Estimated time left: 168.13 seconds.
2025-04-03 15:17:01,107 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:01,124 - Processed 388/500 requests. 99 currently processing. 112 remaining. Estimated time left: 166.75 seconds.
2025-04-03 15:17:01,141 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:01,231 - Processed 389/500 requests. 99 currently processing. 111 remaining. Estimated time left: 164.86 seconds.
2025-04-03 15:17:03,478 - HTTP Request: POST http://loca

2025-04-03 15:17:22,945 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:23,022 - Processed 421/500 requests. 79 currently processing. 79 remaining. Estimated time left: 112.51 seconds.
2025-04-03 15:17:23,127 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:23,178 - Processed 422/500 requests. 78 currently processing. 78 remaining. Estimated time left: 110.85 seconds.
2025-04-03 15:17:23,311 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:23,377 - Processed 423/500 requests. 77 currently processing. 77 remaining. Estimated time left: 109.20 seconds.
2025-04-03 15:17:23,766 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:23,833 - Processed 424/500 requests. 76 currently processing. 76 remaining. Estimated time left: 107.61 seconds.
2025-04-03 15:17:23,855 - HTTP Request: POST http://localhos

2025-04-03 15:17:34,271 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:34,326 - Processed 456/500 requests. 44 currently processing. 44 remaining. Estimated time left: 58.94 seconds.
2025-04-03 15:17:34,327 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:34,334 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:34,441 - Processed 457/500 requests. 43 currently processing. 43 remaining. Estimated time left: 57.49 seconds.
2025-04-03 15:17:34,487 - Processed 458/500 requests. 42 currently processing. 42 remaining. Estimated time left: 56.03 seconds.
2025-04-03 15:17:34,966 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:34,973 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:35,054 - Processed 459/500 requests. 41 currently processing. 41 remain

2025-04-03 15:17:39,543 - Processed 491/500 requests. 9 currently processing. 9 remaining. Estimated time left: 11.29 seconds.
2025-04-03 15:17:39,648 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:39,698 - Processed 492/500 requests. 8 currently processing. 8 remaining. Estimated time left: 10.02 seconds.
2025-04-03 15:17:39,823 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:39,868 - Processed 493/500 requests. 7 currently processing. 7 remaining. Estimated time left: 8.75 seconds.
2025-04-03 15:17:39,891 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:39,948 - Processed 494/500 requests. 6 currently processing. 6 remaining. Estimated time left: 7.49 seconds.
2025-04-03 15:17:39,965 - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-03 15:17:40,021 - Processed 495/500 requests. 5 currently processi

In [213]:
model_output = [""] * len(results)
for result in results:
    model_output[int(result['input'][-1])] = result['output']
Model_choises = model_output
save_to_json_file(Model_choises, '/gemini/code/Abstract_extract/ProcessedData/rdfybk/llama-choise4.json')

数据成功保存到 /gemini/code/Abstract_extract/ProcessedData/rdfybk/llama-choise4.json


In [214]:
Produced_choises = []
for index, choise in enumerate(Model_choises):
    try:
        number_choise = int(choise.split('<最高分序号>')[1].split('</最高分序号>')[0])
    except:
        number_choise = 0
        print(f"第{index}条存在问题，请人工检查")
    Produced_choises.append(number_choise)

第209条存在问题，请人工检查


In [215]:
element_count = Counter(Produced_choises)

# 输出统计结果
for element, count in element_count.items():
    print(f"元素 '{element}' 出现了 {count} 次")

元素 '5' 出现了 135 次
元素 '2' 出现了 142 次
元素 '4' 出现了 177 次
元素 '3' 出现了 45 次
元素 '0' 出现了 1 次
